In [71]:
# importing libraries

In [555]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [556]:
#read dataset

In [1110]:
df=pd.read_excel("test.xlsx")

In [1111]:
df.head()

,venue,team1,team2,toss_winner,toss_decision,winner,season
0,"MA Chidambaram Stadium, Chepauk",RCB,CSK,RCB,BAT,CSK,2024
1,Maharaja Yadavindra Singh Stadium,DC,PBKS,PBKS,BOWL,PBKS,2024
2,Eden Gardens,SRH,KKR,SRH,BOWL,KKR,2024
3,Sawai Mansingh Stadium,LSG,RR,RR,BAT,RR,2024
4,Narendra Modi Stadium,MI,GT,MI,BOWL,GT,2024


In [1112]:
# Given team list
given_team_list = ['MI', 'CSK', 'RCB' , 'DC' , 'GT' , 'LSG' , 'PBKS' , 'RR' , 'KKR' , 'SRH']

# Create a boolean mask for rows where team1 or team2 is not in the given team list
mask = ((df['team1'].isin(given_team_list)) & (df['team2'].isin(given_team_list)))

# Filter out rows based on the mask
filtered_df = df[mask]
df = filtered_df

df = df[df['winner'].notna()] 
df = df[df['winner'] != 'RAIN']

In [1113]:
df.tail()

,venue,team1,team2,toss_winner,toss_decision,winner,season
448,Wankhede Stadium,MI,RR,MI,FIELD,MI,2014
449,Eden Gardens,PBKS,KKR,PBKS,FIELD,KKR,2014
450,Brabourne Stadium,CSK,MI,CSK,FIELD,CSK,2014
451,Wankhede Stadium,CSK,PBKS,CSK,FIELD,PBKS,2014
452,M Chinnaswamy Stadium,KKR,PBKS,KKR,FIELD,KKR,2014


In [1114]:
# checking total rows and columns in the dataset

In [1115]:
print(df.shape)

(419, 7)


In [1116]:
df['winner'].unique()

array(['CSK', 'PBKS', 'KKR', 'RR', 'GT', 'SRH', 'LSG', 'DC', 'MI', 'RCB'],
      dtype=object)

In [1117]:
df['team1'].value_counts()

MI      55
DC      52
SRH     52
RCB     51
PBKS    50
KKR     50
CSK     38
RR      33
GT      26
LSG     12
Name: team1, dtype: int64

In [1118]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
lab = LabelEncoder()

# Perform label encoding on the 'my_column' column
df['team1'] = lab.fit_transform(df['team1'])
df['team2'] = lab.fit_transform(df['team2'])
df['toss_winner'] = lab.fit_transform(df['toss_winner'])
df['toss_decision'] = lab.fit_transform(df['toss_decision'])
df['venue'] = lab.fit_transform(df['venue'])
df['winner'] = lab.fit_transform(df['winner'])
df['season'] = lab.fit_transform(df['season'])

print(df)

     venue  team1  team2  toss_winner  toss_decision  winner  season
0       13      7      0            7              0       0       7
1       14      1      6            6              1       6       7
2        5      9      3            9              1       3       7
3       21      4      8            8              0       8       7
4       16      5      2            5              1       2       7
..     ...    ...    ...          ...            ...     ...     ...
448     23      5      8            5              2       5       0
449      5      6      3            6              2       3       0
450      4      0      5            0              2       0       0
451     23      0      6            0              2       6       0
452     12      3      6            3              2       3       0

[419 rows x 7 columns]


In [1119]:
# Initialize a dictionary to store the win streaks for each team
win_streaks = {team: 0 for team in df['team1'].unique()}

# Initialize the new columns with zeros
df['win_streak_team1'] = 0
df['win_streak_team2'] = 0

# Reset the index of the DataFrame
df = df.reset_index(drop=True)

# Calculate the win streaks
for i in df.index:
    # If it's the start of a new season, reset the win streaks
    if i == 0 or df.loc[i, 'season'] != df.loc[i-1, 'season']:
        win_streaks = {team: 0 for team in df['team1'].unique()}
    
    # Update the win streaks in the DataFrame
    df.at[i, 'win_streak_team1'] = win_streaks[df.loc[i, 'team1']]
    df.at[i, 'win_streak_team2'] = win_streaks[df.loc[i, 'team2']]
    
    # If team1 won the match, increment their win streak
    if df.loc[i, 'winner'] == df.loc[i, 'team1']:
        win_streaks[df.loc[i, 'team1']] += 1
        win_streaks[df.loc[i, 'team2']] -= 1
    # If team2 won the match, increment their win streak
    elif df.loc[i, 'winner'] == df.loc[i, 'team2']:
        win_streaks[df.loc[i, 'team2']] += 1
        win_streaks[df.loc[i, 'team1']] -= 1

df['win_streak_team1'] = df['win_streak_team1'].astype(int)
df['win_streak_team2'] = df['win_streak_team2'].astype(int)
print(df)

     venue  team1  team2  toss_winner  toss_decision  winner  season  \
0       13      7      0            7              0       0       7   
1       14      1      6            6              1       6       7   
2        5      9      3            9              1       3       7   
3       21      4      8            8              0       8       7   
4       16      5      2            5              1       2       7   
..     ...    ...    ...          ...            ...     ...     ...   
414     23      5      8            5              2       5       0   
415      5      6      3            6              2       3       0   
416      4      0      5            0              2       0       0   
417     23      0      6            0              2       6       0   
418     12      3      6            3              2       3       0   

     win_streak_team1  win_streak_team2  
0                   0                 0  
1                   0                 0  
2        

In [1120]:
print(df.columns)

Index(['venue', 'team1', 'team2', 'toss_winner', 'toss_decision', 'winner',
       'season', 'win_streak_team1', 'win_streak_team2'],
      dtype='object')


In [1121]:
from sklearn.preprocessing import LabelEncoder

# Create an instance of LabelEncoder
lab = LabelEncoder()

# Perform label encoding on the 'my_column' column
df['team1'] = lab.fit_transform(df['team1'])
df['team2'] = lab.fit_transform(df['team2'])
df['toss_winner'] = lab.fit_transform(df['toss_winner'])
df['toss_decision'] = lab.fit_transform(df['toss_decision'])
df['venue'] = lab.fit_transform(df['venue'])
df['winner'] = lab.fit_transform(df['winner'])
df['season'] = lab.fit_transform(df['season'])

print(df)

     venue  team1  team2  toss_winner  toss_decision  winner  season  \
0       13      7      0            7              0       0       7   
1       14      1      6            6              1       6       7   
2        5      9      3            9              1       3       7   
3       21      4      8            8              0       8       7   
4       16      5      2            5              1       2       7   
..     ...    ...    ...          ...            ...     ...     ...   
414     23      5      8            5              2       5       0   
415      5      6      3            6              2       3       0   
416      4      0      5            0              2       0       0   
417     23      0      6            0              2       6       0   
418     12      3      6            3              2       3       0   

     win_streak_team1  win_streak_team2  
0                   0                 0  
1                   0                 0  
2        

In [1122]:
import seaborn as sns
# Calculate the correlation matrix
corr_matrix = df.corr()

# Display the correlation matrix
print(corr_matrix)

                     venue     team1     team2  toss_winner  toss_decision  \
venue             1.000000  0.015212  0.372577     0.132193      -0.085119   
team1             0.015212  1.000000 -0.135912     0.448283      -0.036604   
team2             0.372577 -0.135912  1.000000     0.434602       0.000927   
toss_winner       0.132193  0.448283  0.434602     1.000000      -0.062867   
toss_decision    -0.085119 -0.036604  0.000927    -0.062867       1.000000   
winner            0.182528  0.377160  0.494942     0.498672       0.009624   
season            0.036199 -0.032429 -0.064513    -0.000030      -0.196573   
win_streak_team1  0.001823 -0.113800 -0.042883    -0.181215       0.065306   
win_streak_team2  0.051025 -0.003778 -0.074532    -0.033170      -0.065942   

                    winner    season  win_streak_team1  win_streak_team2  
venue             0.182528  0.036199          0.001823          0.051025  
team1             0.377160 -0.032429         -0.113800         -0.003

In [1123]:
X = df.drop(columns=['winner','season'],axis=1)
Y = df['winner']

In [1124]:
print(X)

     venue  team1  team2  toss_winner  toss_decision  win_streak_team1  \
0       13      7      0            7              0                 0   
1       14      1      6            6              1                 0   
2        5      9      3            9              1                 0   
3       21      4      8            8              0                 0   
4       16      5      2            5              1                 0   
..     ...    ...    ...          ...            ...               ...   
414     23      5      8            5              2                 4   
415      5      6      3            6              2                 3   
416      4      0      5            0              2                 0   
417     23      0      6            0              2                 1   
418     12      3      6            3              2                 7   

     win_streak_team2  
0                   0  
1                   0  
2                   0  
3              

In [1125]:
print(Y)

0      0
1      6
2      3
3      8
4      2
      ..
414    5
415    3
416    0
417    6
418    3
Name: winner, Length: 419, dtype: int64


In [1126]:
X_train,X_test,y_train,y_test = train_test_split(X,Y, test_size=0.2, random_state=1) #stratify=Y

In [1127]:
#knn

In [1128]:
knn = KNeighborsClassifier(n_neighbors=5)

In [1129]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [1130]:
y_pred = knn.predict(X_test)

In [1131]:
accuracy = accuracy_score(y_pred, y_test)
print(f"Accuracy score of test data: {accuracy:.2f}")

Accuracy score of test data: 0.36


In [1132]:
train_pred = knn.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))

Accuracy score of train data = 0.5701492537313433


In [1133]:
#Decison Tree

In [1146]:
tree_classifier = DecisionTreeClassifier(max_depth=11, criterion='gini')
tree_classifier.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=11)

In [1147]:
train_pred = tree_classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))

Accuracy score of train data = 0.8776119402985074


In [1148]:
y_pred = tree_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of test data: {accuracy:.2f}")

Accuracy of test data: 0.48


In [1149]:
#random forest

In [1150]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=9, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(max_depth=9, random_state=42)

In [1151]:
train_pred = rf_classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))

Accuracy score of train data = 0.9611940298507463


In [1152]:
y_pred = rf_classifier.predict(X_test)

In [1153]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of test data: {accuracy:.2f}")

Accuracy of test data: 0.58


In [1154]:
# SVM Poly

In [1155]:
classifier = svm.SVC(kernel='poly' , degree =10 , gamma='scale')

In [1156]:
#Model training using test data

In [1157]:
classifier.fit(X_train, y_train)

SVC(degree=10, kernel='poly')

In [1158]:
# Accuracy score on train data

In [1159]:
train_pred = classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)

In [1160]:
print("Accuracy score of train data = {}".format(accuracy))

Accuracy score of train data = 0.6746268656716418


In [1161]:
# Accuracy score on test data

In [1162]:
test_pred = classifier.predict(X_test)
accuracy2 = accuracy_score(test_pred, y_test)

In [1163]:
print('Accuracy score on test data = {}'.format(accuracy2))

Accuracy score on test data = 0.2976190476190476


In [1164]:
#SVM RBF

In [1165]:
classifier = svm.SVC(kernel='rbf', gamma='scale')

In [1166]:
classifier.fit(X_train, y_train)

SVC()

In [1167]:
train_pred = classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)

In [1168]:
print("Accuracy score of train data = {}".format(accuracy))

Accuracy score of train data = 0.43283582089552236


In [1169]:
test_pred = classifier.predict(X_test)
accuracy2 = accuracy_score(test_pred, y_test)

In [1170]:
print('Accuracy score on test data = {}'.format(accuracy2))

Accuracy score on test data = 0.4166666666666667


In [1171]:
#lightGBM

In [1172]:
lgbm_classifier = LGBMClassifier(boosting_type='gbdt', num_leaves=9, learning_rate=0.1, n_estimators=200)
lgbm_classifier.fit(X_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000088 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 83
[LightGBM] [Info] Number of data points in the train set: 335, number of used features: 7
[LightGBM] [Info] Start training from score -2.176544
[LightGBM] [Info] Start training from score -2.317623
[LightGBM] [Info] Start training from score -3.041542
[LightGBM] [Info] Start training from score -1.985489
[LightGBM] [Info] Start training from score -3.416235
[LightGBM] [Info] Start training from score -1.942930
[LightGBM] [Info] Start training from score -2.287770
[LightGBM] [Info] Start training from score -2.230612
[LightGBM] [Info] Start training from score -2.412933
[LightGBM] [Info] Start training from score -2.052930
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fu

LGBMClassifier(n_estimators=200, num_leaves=9)

In [1173]:
train_pred = lgbm_classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))

Accuracy score of train data = 0.9970149253731343


In [1174]:
y_pred = lgbm_classifier.predict(X_test)
accuracy = accuracy_score(y_pred,y_test)
print(f"Accuracy score of test data: {accuracy:.2f}")


Accuracy score of test data: 0.64


In [1175]:
#Naive Bayes

In [1176]:
nb_classifier = GaussianNB()
nb_classifier.fit(X_train, y_train)

GaussianNB()

In [1177]:
train_pred = nb_classifier.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))

Accuracy score of train data = 0.4208955223880597


In [1178]:
y_pred = nb_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy score of test data: {accuracy:.2f}")

Accuracy score of test data: 0.42


In [1179]:
#CatBoost

In [1180]:
model = CatBoostClassifier(iterations=100, depth=3, learning_rate=0.1, loss_function='MultiClass', verbose=False)
model.fit(X_train, y_train)

In [1181]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of test data: {accuracy:.2f}")

Accuracy of test data: 0.57


In [1182]:
train_pred = model.predict(X_train)
accuracy = accuracy_score(train_pred, y_train)
print("Accuracy score of train data = {}".format(accuracy))

Accuracy score of train data = 0.6716417910447762


In [1183]:
# Create a DataFrame for the new data
data = {
    'venue': ['Narendra Modi Stadium'],
    'team1': ['RCB'],
    'team2': ['GT'],
    'toss_winner': ['RCB'],
    'toss_decision' : ['BOWL']
}
df1 = pd.DataFrame(data)

df1['win_streak_team1'] = df1['team1'].map(win_streaks)
df1['win_streak_team2'] = df1['team2'].map(win_streaks)

# Assuming you've used LabelEncoder during training, you need to transform your new data as well
encoder = LabelEncoder()

# Fit and transform the data
df1['venue'] = encoder.fit_transform(df1['venue'])
df1['team1'] = encoder.fit_transform(df1['team1'])
df1['team2'] = encoder.fit_transform(df1['team2'])
df1['toss_winner'] = encoder.fit_transform(df1['toss_winner'])
df1['toss_decision'] = encoder.fit_transform(df1['toss_decision'])
df1 = df1.fillna(0)

# Use the model to predict the winner
predictions = rf_classifier.predict(df1)

# Inverse transform the predicted winner
decoded_predictions = team_encoder.inverse_transform(predictions)

print(decoded_predictions[0])

GT
